#Aim

To find the heavily correlated f&o stocks.

I'll be using Google Colab for the project and recommend the same if you want to follow along!

---

For this project, I'll be using the Yahoo Finance API.

In [42]:
# Installing the yfinance module to the Jupyter notebook
!pip install yfinance

In [43]:
# Importing all the neccesary modules
import yfinance as yf
import pandas as pd
import time

I've stored all the stock names in a `.txt` file. So before we extract all the stock prices for each year, we'll have to convert this file into a `.csv` file.

In [44]:
# Code to convert the .txt file to a .csv file
stocks = []
file = open("/content/f&o stocks.txt", "r")

for line in file.readlines():
  stocks.append(line.split("\t"))

df = pd.DataFrame(stocks, columns = ["Stock Name", "Stock Symbol"])

symbol_list = []
for symbol in df.loc[:, "Stock Symbol"]:
  symbol_list.append(symbol.replace("\n", ".NS")) # Here am adding .NS to each symbol as he represents that the symbol belongs to NSE

df.loc[:, "Stock Symbol"] = symbol_list
df.to_csv("F&O stocks.csv", index = False)

With the above code finishing it's execution, you'll see a `.csv` file with the name `F&O stocks.csv` in the Files folder to your left.

---

Next, we copy the path of the `F&O stocks.csv` file, open it as a pandas dataframe, and store the data in the `Stock Symbol`column as a variable

In [45]:
# Opening the F&O stocks.csv file as a pandas dtaframe
fNo_stocks_df = pd.read_csv("/content/F&O stocks.csv")

# Storing all the symbols in one variable
fNo_stocks = fNo_stocks_df.loc[:, "Stock Symbol"]

Next, we get the `closing` prices of the stocks since their listing and store it in a list.

In [47]:
# Downloading the price closes since the listing of the stock
stocks_data = []

for stock in fNo_stocks:
  print(stock) # I am printing the stock name here just to know which stock is the program currently on
  stocks_data.append(yf.download(stock)["Close"])

AARTIIND.NS
[*********************100%***********************]  1 of 1 completed
ABB.NS
[*********************100%***********************]  1 of 1 completed
ABBOTINDIA.NS
[*********************100%***********************]  1 of 1 completed
ABCAPITAL.NS
[*********************100%***********************]  1 of 1 completed
ABFRL.NS
[*********************100%***********************]  1 of 1 completed
ACC.NS
[*********************100%***********************]  1 of 1 completed
ADANIENT.NS
[*********************100%***********************]  1 of 1 completed
ADANIPORTS.NS
[*********************100%***********************]  1 of 1 completed
ALKEM.NS
[*********************100%***********************]  1 of 1 completed
AMARAJABAT.NS
[*********************100%***********************]  1 of 1 completed
AMBUJACEM.NS
[*********************100%***********************]  1 of 1 completed
APLLTD.NS
[*********************100%***********************]  1 of 1 completed
APOLLOHOSP.NS
[*********************10

Here we can't get the prices for ZYDUSLIFE as it was probably delisted from the exchange.

Here we get the different dates for which the prices were available for the stocks. We are doing this because we'll be storing prices for each symbol on a daily basis.

In [48]:
# Storing all the dates in a set
dates = set()

for stock_data in stocks_data:
  for date in stock_data.index:
    dates.add(date)

dates = list(dates)
dates.sort()

Now we will alot the prices for each symbol to the day it belongs to.

In [49]:
# Creating a prices dictionary and alotting prices to it
prices = {}

for date in dates:
  prices[date] = []

for date in dates:
  for stock_data in stocks_data:
    try:
      prices[date].append(stock_data[date])
    except:
      prices[date].append(None)

---

Now that we have the prices stored on a daily basis, we can convert the dictionary they are stored in into a pandas dataframe.

In [50]:
# Converting dictionary to dataframe
fNo_stocks_close_df = pd.DataFrame(data = list(prices.values()), index = list(prices.keys()), columns = fNo_stocks)

Now we'll convert the dataframe we got into a correlation dataframe.

In [51]:
# Printing the correlation dataframe
fNo_stocks_corr_df = round(fNo_stocks_close_df.corr(), 2)
fNo_stocks_corr_df

Stock Symbol,AARTIIND.NS,ABB.NS,ABBOTINDIA.NS,ABCAPITAL.NS,ABFRL.NS,ACC.NS,ADANIENT.NS,ADANIPORTS.NS,ALKEM.NS,AMARAJABAT.NS,...,TRENT.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WHIRLPOOL.NS,WIPRO.NS,ZEEL.NS
Stock Symbol,,,,,,,,,,,,,,,,,,,,,
AARTIIND.NS,1.00,0.68,0.96,-0.23,0.65,0.71,0.86,0.92,0.93,0.63,...,0.98,0.82,0.79,0.91,0.89,0.32,0.95,0.90,0.91,0.39
ABB.NS,0.68,1.00,0.66,0.36,0.75,0.88,0.56,0.86,0.53,0.76,...,0.69,0.76,0.79,0.81,0.82,0.68,0.82,0.72,0.76,0.72
ABBOTINDIA.NS,0.96,0.66,1.00,-0.55,0.55,0.71,0.76,0.86,0.89,0.69,...,0.96,0.80,0.78,0.90,0.86,0.28,0.93,0.94,0.87,0.40
ABCAPITAL.NS,-0.23,0.36,-0.55,1.00,0.03,0.35,0.01,0.20,-0.15,0.38,...,-0.23,0.84,-0.04,0.11,0.13,0.79,0.04,-0.43,-0.02,0.75
ABFRL.NS,0.65,0.75,0.55,0.03,1.00,0.60,0.59,0.65,0.46,0.26,...,0.63,0.55,0.74,0.67,0.73,0.14,0.69,0.54,0.59,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VEDL.NS,0.32,0.68,0.28,0.79,0.14,0.64,0.37,0.21,0.40,0.36,...,0.36,0.47,-0.06,0.44,0.44,1.00,0.49,0.34,0.64,0.47
VOLTAS.NS,0.95,0.82,0.93,0.04,0.69,0.82,0.80,0.96,0.90,0.73,...,0.96,0.90,0.81,0.95,0.94,0.49,1.00,0.93,0.92,0.56
WHIRLPOOL.NS,0.90,0.72,0.94,-0.43,0.54,0.76,0.60,0.86,0.77,0.80,...,0.90,0.91,0.83,0.92,0.93,0.34,0.93,1.00,0.80,0.59


I wanted to get the heatmap for this dataframe but cause the dataframe has 199 rows and columns, getting a heatmap of the correct size (with correlational coefficient printed in a readable format) is not possible.

Finally we have the correlations of each stock stored in a dataframe.

You can try out many different methods to get the most correlated stocks from this dataframe, and can research about the reasons behind this correlation.

---

#END OF THE PROJECT

If you want to save the dataframe as a `.csv` file uncomment the code below and run it.

In [52]:
# fNo_stocks_corr_df.to_excel("F&O Stocks Correlation.xlsx")